# Wildfire Impact Estimate in Pueblo, CO
## Notebook 2: Getting AQI Data from the EPA

In this notebook, I pull Air Quality Index (AQI) data provided by the US EPA quantifying how six different airborne contaminants affect air quality. They are:

- Acceptable PM2.5 AQI & Speciation Mass
- Carbon monoxide
- Nitrogen dioxide (NO2)
- Ozone
- PM10 Total 0-10um STP
- PM2.5 - Local Conditions
- Sulfur dioxide

I access the data via the [EPA's AQS REST API](https://aqs.epa.gov/aqsweb/documents/data_api.html).

## Environment Setup and Constant Definition

### ATTRIBUTION
The code and comments below are adopted, with light modifications, from Dr. David McDonald,
who provided them for use in DATA 512, a course in the University of Washington MS of Data
Science Program. The code is provided and utilized here under the [Creative Commons CC-BY license.](https://creativecommons.org/licenses/by/4.0/)

In [1]:
import json
import time
import requests
from tqdm import tqdm
from dotenv import load_dotenv
import os

In [2]:
#    CONSTANTS

USERNAME = "dvogler@uw.edu"
load_dotenv()
APIKEY = os.getenv("APIKEY")

#    This is the root of all AQS API URLs
#
API_REQUEST_URL = 'https://aqs.epa.gov/data/api'


#    These are some of the 'actions' we can ask the API to take or requests that we can make of the API

#    Sign-up request - generally only performed once - unless you lose your key
API_ACTION_SIGNUP = '/signup?email={email}'

#    List actions provide information on API parameter values that are required by some other actions/requests
API_ACTION_LIST_CLASSES = '/list/classes?email={email}&key={key}'
API_ACTION_LIST_PARAMS = '/list/parametersByClass?email={email}&key={key}&pc={pclass}'
API_ACTION_LIST_SITES = '/list/sitesByCounty?email={email}&key={key}&state={state}&county={county}'

#    Monitor actions are requests for monitoring stations that meet specific criteria
API_ACTION_MONITORS_COUNTY = '/monitors/byCounty?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&state={state}&county={county}'
API_ACTION_MONITORS_BOX = '/monitors/byBox?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&minlat={minlat}&maxlat={maxlat}&minlon={minlon}&maxlon={maxlon}'

#    Summary actions are requests for summary data. These are for daily summaries
API_ACTION_DAILY_SUMMARY_COUNTY = '/dailyData/byCounty?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&state={state}&county={county}'
API_ACTION_DAILY_SUMMARY_BOX = '/dailyData/byBox?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&minlat={minlat}&maxlat={maxlat}&minlon={minlon}&maxlon={maxlon}'

#    It is always nice to be respectful of a free data resource.
#    We're going to observe a 100 requests per minute limit - which is fairly nice
# Assuming roughly 2ms latency on the API and network
API_LATENCY_ASSUMED = 0.002
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

#    This is a template that covers most of the parameters for the actions we might take, from the set of actions
#    above. In the examples below, most of the time parameters can either be supplied as individual values to a
#    function - or they can be set in a copy of the template and passed in with the template.

AQS_REQUEST_TEMPLATE = {
    "email":      "",
    "key":        "",
    "state":      "",     # the two digit state FIPS # as a string
    "county":     "",     # the three digit county FIPS # as a string
    "begin_date": "",     # the start of a time window in YYYYMMDD format
    # the end of a time window in YYYYMMDD format, begin_date and end_date must be in the same year
    "end_date":   "",
    "minlat":    0.0,
    "maxlat":    0.0,
    "minlon":    0.0,
    "maxlon":    0.0,
    "param":     "",     # a list of comma separated 5 digit codes, max 5 codes requested
    "pclass":    ""      # parameter class is only used by the List calls
}

#   Gaseous AQI pollutants CO, SO2, NO2, and O2
AQI_PARAMS_GASEOUS = "42101,42401,42602,44201"

#   Particulate AQI pollutants PM10, PM2.5, and Acceptable PM2.5
AQI_PARAMS_PARTICULATES = "81102,88101,88502"


CITY_LOCATIONS = {
    "pueblo":       {"city": "Pueblo",
                     "county": "Pueblo",
                     "state": ["Colorado", "CO"],
                     "fips": "08101",
                     "latlon": [38.2706, -104.6101]
                     }
}

## Defining Functions for API Calls

I will pull daily air quality data, then aggregate it to get yearly figures.

The goal of this analysis is to get data from 1961-2021; Pueblo, CO itself did not have measuring stations for all of the relevant contaminants in all of those years. In order to get as much data as possible, I will draw a 50-mile bounding box around Pueblo and ask the EPA for AQI data for any measurement station in that region, treating this as a proxy for air quality in Pueblo.

The functions below respectively:
- draw that bounding box
- allow me to check which stations (performing which measurements) are available
- pull daily data from the stations that are available

In [4]:
#
#   These are rough estimates for creating bounding boxes based on a city location
#   You can find these rough estimates on the USGS website:
#   https://www.usgs.gov/faqs/how-much-distance-does-a-degree-minute-and-second-cover-your-maps
#
# This is about 25 miles of latitude in decimal degrees
LAT_25MILES = 25.0 * (1.0/69.0)
# This is about 25 miles of longitude in decimal degrees
LON_25MILES = 25.0 * (1.0/54.6)

#   Compute rough estimates for a bounding box around a given place
#   The bounding box is scaled in 50 mile increments. That is, the bounding box will have sides that
#   are rough multiples of 50 miles, with the center of the box around the indicated place.
#   The scale parameter determines the scale (size) of the bounding box


def bounding_latlon(place=None, scale=1.0):
    minlat = place['latlon'][0] - float(scale) * LAT_25MILES
    maxlat = place['latlon'][0] + float(scale) * LAT_25MILES
    minlon = place['latlon'][1] - float(scale) * LON_25MILES
    maxlon = place['latlon'][1] + float(scale) * LON_25MILES
    return [minlat, maxlat, minlon, maxlon]

#
#    This implements the monitors request. This requests monitoring stations. This can be done by state, county, or bounding box.
#    Like the two other functions, this can be called with a mixture of a defined parameter dictionary, or with function
#    parameters. If function parameters are provided, those take precedence over any parameters from the request template.
#
def request_monitors(email_address=None, key=None, param=None,
                     begin_date=None, end_date=None, fips=None,
                     endpoint_url=API_REQUEST_URL,
                     endpoint_action=API_ACTION_MONITORS_COUNTY,
                     request_template=AQS_REQUEST_TEMPLATE,
                     headers=None):

    #  This prioritizes the info from the call parameters - not what's already in the template
    if email_address:
        request_template['email'] = email_address
    if key:
        request_template['key'] = key
    if param:
        request_template['param'] = param
    if begin_date:
        request_template['begin_date'] = begin_date
    if end_date:
        request_template['end_date'] = end_date
    if fips and len(fips) == 5:
        request_template['state'] = fips[:2]
        request_template['county'] = fips[2:]

    # Make sure there are values that allow us to make a call - these are always required
    if not request_template['email']:
        raise Exception(
            "Must supply an email address to call 'request_monitors()'")
    if not request_template['key']:
        raise Exception("Must supply a key to call 'request_monitors()'")
    if not request_template['param']:
        raise Exception(
            "Must supply param values to call 'request_monitors()'")
    if not request_template['begin_date']:
        raise Exception(
            "Must supply a begin_date to call 'request_monitors()'")
    if not request_template['end_date']:
        raise Exception("Must supply an end_date to call 'request_monitors()'")
    # Note we're not validating FIPS fields because not all of the monitors actions require the FIPS numbers

    # compose the request
    request_url = endpoint_url+endpoint_action.format(**request_template)

    # make the request
    try:
        # Wait first, to make sure we don't exceed a rate limit in the situation where an exception occurs
        # during the request processing - throttling is always a good practice with a free data source
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

#
#    This implements the daily summary request. Daily summary provides a daily summary value for each sensor being requested
#    from the start date to the end date.
#
#    Like the two other functions, this can be called with a mixture of a defined parameter dictionary, or with function
#    parameters. If function parameters are provided, those take precedence over any parameters from the request template.
#
def request_daily_summary(email_address=None, key=None, param=None,
                          begin_date=None, end_date=None, fips=None,
                          endpoint_url=API_REQUEST_URL,
                          endpoint_action=API_ACTION_DAILY_SUMMARY_COUNTY,
                          request_template=AQS_REQUEST_TEMPLATE,
                          headers=None):

    #  This prioritizes the info from the call parameters - not what's already in the template
    if email_address:
        request_template['email'] = email_address
    if key:
        request_template['key'] = key
    if param:
        request_template['param'] = param
    if begin_date:
        request_template['begin_date'] = begin_date
    if end_date:
        request_template['end_date'] = end_date
    if fips and len(fips) == 5:
        request_template['state'] = fips[:2]
        request_template['county'] = fips[2:]

    # Make sure there are values that allow us to make a call - these are always required
    if not request_template['email']:
        raise Exception(
            "Must supply an email address to call 'request_daily_summary()'")
    if not request_template['key']:
        raise Exception("Must supply a key to call 'request_daily_summary()'")
    if not request_template['param']:
        raise Exception(
            "Must supply param values to call 'request_daily_summary()'")
    if not request_template['begin_date']:
        raise Exception(
            "Must supply a begin_date to call 'request_daily_summary()'")
    if not request_template['end_date']:
        raise Exception(
            "Must supply an end_date to call 'request_daily_summary()'")
    # Note we're not validating FIPS fields because not all of the daily summary actions require the FIPS numbers

    # compose the request
    request_url = endpoint_url+endpoint_action.format(**request_template)

    # make the request
    try:
        # Wait first, to make sure we don't exceed a rate limit in the situation where an exception occurs
        # during the request processing - throttling is always a good practice with a free data source
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

## Pulling AQI Data Around Pueblo, CO

There are two groups of contaminants: particulate and gaseous. 

First, I check which stations around Pueblo measure particulate AQIs.

In [5]:
#
#    Create a copy of the AQS_REQUEST_TEMPLATE
#
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY

# same particulate request as the one above
request_data['param'] = AQI_PARAMS_PARTICULATES
#

#   50 mile box
bbox = bounding_latlon(CITY_LOCATIONS['pueblo'], scale=2.0)

#   put our bounding box into the request_data
request_data['minlat'] = bbox[0]
request_data['maxlat'] = bbox[1]
request_data['minlon'] = bbox[2]
request_data['maxlon'] = bbox[3]

#
#   we need to change the action for the API from the default to the bounding box - same recent date for now
response = request_monitors(request_template=request_data, 
                            begin_date="19610101", 
                            end_date="20211231",
                            endpoint_action=API_ACTION_MONITORS_BOX)
#
#
#
if response["Header"][0]['status'] == "Success":
    print(json.dumps(response['Data'], indent=4))
else:
    print(json.dumps(response, indent=4))

[
    {
        "state_code": "08",
        "county_code": "101",
        "site_number": "0001",
        "parameter_code": "81102",
        "poc": 2,
        "parameter_name": "PM10 Total 0-10um STP",
        "open_date": "1987-10-01",
        "close_date": "1990-11-01",
        "concurred_exclusions": null,
        "dominant_source": null,
        "measurement_scale": null,
        "measurement_scale_def": null,
        "monitoring_objective": "UNKNOWN",
        "last_method_code": null,
        "last_method_description": " - ",
        "last_method_begin_date": null,
        "naaqs_primary_monitor": null,
        "qa_primary_monitor": null,
        "monitor_type": null,
        "networks": null,
        "monitoring_agency_code": "0240",
        "monitoring_agency": "Colorado Department of Public Health And Environment",
        "si_id": 1932,
        "latitude": 38.266392,
        "longitude": -104.61053,
        "datum": "WGS84",
        "lat_lon_accuracy": 0.0,
        "elevation":

By analyzing the state/county codes below, we can see that Pueblo, CO (FIPS: 08101) itself does a good job of measuring particulate concentrations. Most of the FIPS codes below are for Pueblo itself, not cities around it.

In [6]:
for station in response["Data"]:
    print(station)

{'state_code': '08', 'county_code': '101', 'site_number': '0001', 'parameter_code': '81102', 'poc': 2, 'parameter_name': 'PM10 Total 0-10um STP', 'open_date': '1987-10-01', 'close_date': '1990-11-01', 'concurred_exclusions': None, 'dominant_source': None, 'measurement_scale': None, 'measurement_scale_def': None, 'monitoring_objective': 'UNKNOWN', 'last_method_code': None, 'last_method_description': ' - ', 'last_method_begin_date': None, 'naaqs_primary_monitor': None, 'qa_primary_monitor': None, 'monitor_type': None, 'networks': None, 'monitoring_agency_code': '0240', 'monitoring_agency': 'Colorado Department of Public Health And Environment', 'si_id': 1932, 'latitude': 38.266392, 'longitude': -104.61053, 'datum': 'WGS84', 'lat_lon_accuracy': 0.0, 'elevation': 1426.0, 'probe_height': None, 'pl_probe_location': None, 'local_site_name': None, 'address': '151 CENTRAL MAIN PUEBLO HEALTH DEPT.', 'state_name': 'Colorado', 'county_name': 'Pueblo', 'city_name': 'Pueblo', 'cbsa_code': '39380', '

For gaseous components, the story is different.

Although searching for gaseous AQI components for Pueblo, CO yields results, they are only for a few years. The remainder of the output (cell 8) has FIPS codes that are outside of Pueblo, within the 100mi bounding box. So to get data covering as many years as possible, I treat the results within the 100mi box around Pueblo as proxies of air quality in Pueblo.

In [7]:
#
#    Create a copy of the AQS_REQUEST_TEMPLATE
#
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY
# same particulate request as the one abover
request_data['param'] = AQI_PARAMS_GASEOUS
#

#   50 mile box
bbox = bounding_latlon(CITY_LOCATIONS['pueblo'], scale=2.0)

#   put our bounding box into the request_data
request_data['minlat'] = bbox[0]
request_data['maxlat'] = bbox[1]
request_data['minlon'] = bbox[2]
request_data['maxlon'] = bbox[3]

#
#   we need to change the action for the API from the default to the bounding box - same recent date for now
response = request_monitors(request_template=request_data, 
                            begin_date="19610101", 
                            end_date="20211231",
                            endpoint_action=API_ACTION_MONITORS_BOX)
#
#
#
if response["Header"][0]['status'] == "Success":
    print(json.dumps(response['Data'], indent=4))
else:
    print(json.dumps(response, indent=4))

[
    {
        "state_code": "08",
        "county_code": "041",
        "site_number": "2001",
        "parameter_code": "44201",
        "poc": 1,
        "parameter_name": "Ozone",
        "open_date": "1981-06-22",
        "close_date": "1988-05-24",
        "concurred_exclusions": null,
        "dominant_source": "AREA",
        "measurement_scale": "URBAN SCALE",
        "measurement_scale_def": "4 KM TO 50 KM",
        "monitoring_objective": "HIGHEST CONCENTRATION",
        "last_method_code": "014",
        "last_method_description": "INSTRUMENTAL - CHEMILUMINESCENCE",
        "last_method_begin_date": "1981-06-22",
        "naaqs_primary_monitor": null,
        "qa_primary_monitor": null,
        "monitor_type": null,
        "networks": null,
        "monitoring_agency_code": "0240",
        "monitoring_agency": "Colorado Department of Public Health And Environment",
        "si_id": 1784,
        "latitude": 38.758606,
        "longitude": -104.753861,
        "datum": "WG

In [8]:
for station in response["Data"]:
    print(station)

{'state_code': '08', 'county_code': '041', 'site_number': '2001', 'parameter_code': '44201', 'poc': 1, 'parameter_name': 'Ozone', 'open_date': '1981-06-22', 'close_date': '1988-05-24', 'concurred_exclusions': None, 'dominant_source': 'AREA', 'measurement_scale': 'URBAN SCALE', 'measurement_scale_def': '4 KM TO 50 KM', 'monitoring_objective': 'HIGHEST CONCENTRATION', 'last_method_code': '014', 'last_method_description': 'INSTRUMENTAL - CHEMILUMINESCENCE', 'last_method_begin_date': '1981-06-22', 'naaqs_primary_monitor': None, 'qa_primary_monitor': None, 'monitor_type': None, 'networks': None, 'monitoring_agency_code': '0240', 'monitoring_agency': 'Colorado Department of Public Health And Environment', 'si_id': 1784, 'latitude': 38.758606, 'longitude': -104.753861, 'datum': 'WGS84', 'lat_lon_accuracy': 0.0, 'elevation': 1756.0, 'probe_height': 4.0, 'pl_probe_location': None, 'local_site_name': 'PINELLO RANCH', 'address': 'PINELLO RANCH', 'state_name': 'Colorado', 'county_name': 'El Paso',

Daily AQI data can only be pulled one year at a time.

The code below:
- draws a 100mi box around Pueblo
- gets data about the AQI from gaseous pollutants inside of that box
- gets data about the AQI from particulate pollutants inside of that box
- saves the results to the [`intermediate_data`](../intermediate_data/) folder

In [14]:
# best practice to make a copy of the template and "start fresh" rather
# than modify the template
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY

AQI_PARAM_CLASS = "AQI POLLUTANTS"
request_data['pclass'] = AQI_PARAM_CLASS

# draw 50 mile box  in each direction around Pueblo (100mi length)
# scale 1.0 does this at 25 mi; shifting scale to 2.0 expands it
bbox = bounding_latlon(CITY_LOCATIONS['pueblo'], scale=2.0)

#   put our bounding box into the request_data
request_data['minlat'] = bbox[0]
request_data['maxlat'] = bbox[1]
request_data['minlon'] = bbox[2]
request_data['maxlon'] = bbox[3]

START_YEAR = 1960 # not inclusive
END_YEAR = 2023

# use tqdm in the for loop for a nice progress bar
# for loop counts backwards; this ensures I catch 
# problems early, since more recent years are likelier to
# have data
for year in tqdm(range(END_YEAR, START_YEAR, - 1)): 
    
    print(f"Attempting to pull data for {year}")
    # request daily summary data for the month of July in 2021
    gaseous_aqi = request_daily_summary(request_template=request_data, 
                                        begin_date=f"{year}0101", 
                                        end_date=f"{year}1231", 
                                        param=AQI_PARAMS_GASEOUS,
                                        endpoint_action=API_ACTION_DAILY_SUMMARY_BOX)
    print("Response for the gaseous pollutants ...")
    #
    if gaseous_aqi["Header"][0]['status'] == "Success":
        print(f"Succeeded with gaseous data pull for year {year}")
        with open(f"../intermediate_data/gaseous_aqi_{year}.json", "w") as file:
            json.dump(gaseous_aqi,file, indent = 4)
    elif gaseous_aqi["Header"][0]['status'].startswith("No data "):
        print("Looks like the response generated no data. You might take a closer look at your request and the response data.")
    else:
        print(f"Warning: something unusual happen in data for {year}. Check saved json.")
        with open(f"../intermediate_data/gaseous_aqi_{year}.json", "w") as file:
            json.dump(gaseous_aqi,file, indent = 4)
    
    # request daily summary data for the month of July in 2021
    particulate_aqi = request_daily_summary(request_template=request_data, 
                                            begin_date=f"{year}0101", 
                                            end_date=f"{year}1231",
                                            param=AQI_PARAMS_PARTICULATES, 
                                            endpoint_action=API_ACTION_DAILY_SUMMARY_BOX)
    print("Response for the particulate pollutants ...")
    #
    if particulate_aqi["Header"][0]['status'] == "Success":
        print(f"Succeeded with particulate data pull for year {year}")
        with open(f"../intermediate_data/particulate_aqi_{year}.json", "w") as file:
            json.dump(particulate_aqi,file, indent = 4)
    elif particulate_aqi["Header"][0]['status'].startswith("No data "):
        print("Looks like the response generated no data. You might take a closer look at your request and the response data.")
    else:
        print(f"Warning: something unusual happen in data for {year}. Check saved json.")
        with open(f"../intermediate_data/particulate_aqi_{year}.json", "w") as file:
            json.dump(particulate_aqi,file, indent = 4)


  0%|          | 0/63 [00:00<?, ?it/s]

Attempting to pull data for 2023
Response for the gaseous pollutants ...
Succeeded with gaseous data pull for year 2023


  2%|▏         | 1/63 [02:36<2:41:32, 156.33s/it]

Response for the particulate pollutants ...
Succeeded with particulate data pull for year 2023
Attempting to pull data for 2022
Response for the gaseous pollutants ...
Succeeded with gaseous data pull for year 2022


  3%|▎         | 2/63 [05:13<2:39:08, 156.54s/it]

...

Response for the particulate pollutants ...
Looks like the response generated no data. You might take a closer look at your request and the response data.
Attempting to pull data for 1962
Response for the gaseous pollutants ...
Looks like the response generated no data. You might take a closer look at your request and the response data.


 98%|█████████▊| 62/63 [2:40:03<02:31, 151.12s/it]

Response for the particulate pollutants ...
Looks like the response generated no data. You might take a closer look at your request and the response data.
Attempting to pull data for 1961
Response for the gaseous pollutants ...
Looks like the response generated no data. You might take a closer look at your request and the response data.


100%|██████████| 63/63 [2:42:34<00:00, 154.84s/it]

Response for the particulate pollutants ...
Looks like the response generated no data. You might take a closer look at your request and the response data.


Now that I have AQI data, I will develop my own metric of smoke intensity in [Notebook 3](./combining_distance_and_aqi.ipynb) and compare it to AQI.